In [1]:
from utils import predict
import importlib
import nibabel as nib
from monai.transforms import Compose, LoadImaged, Orientationd, AsChannelLastd, ScaleIntensityd, Resized, AsChannelFirstd
from monai.transforms import Compose, LoadImage, Orientation, AsChannelLast, ScaleIntensity, Resize, AsChannelFirst, SplitDimd, ToNumpyd
import yaml
from yaml import Loader
import pandas as pd
import csv

2023-09-02 12:49:15.453073: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# importlib.reload(utils)
# utils.discover_files('/home/raffi/Projects/DeepDicomSort_MONAI/data/Brain-Tumor-Progression')

In [ ]:
i = 5
def gen(i):
    for k in range(i):
        yield k

In [ ]:
generator = gen(i)

In [ ]:
generator.__next__()

In [17]:
path = "data/Brain-Tumor-Progression_nifti/B_Ax_T1_+C/PGBM-005.nii"
image = nib.load(path)
image.shape

(280, 320, 22)

In [47]:
transforms = Compose([
        LoadImaged(keys=["image"], reader="NibabelReader", ensure_channel_first=True),
        Orientationd(keys=["image"], axcodes="RAS"),
        Resized(keys=["image"], spatial_size=[256, 256, 25], mode="trilinear"),
        SplitDimd(keys=["image"], dim=3, keepdim=False),
        ScaleIntensityd(keys=[f"image_{i}" for i in range(25)], minv=0, maxv=1, channel_wise=True),
        ToNumpyd(keys=[f"image_{i}" for i in range(25)])
    ])
output = transforms({"image": path})
display(output.keys())
output["image_0"].shape

dict_keys(['image', 'image_meta_dict', 'image_0', 'image_1', 'image_2', 'image_3', 'image_4', 'image_5', 'image_6', 'image_7', 'image_8', 'image_9', 'image_10', 'image_11', 'image_12', 'image_13', 'image_14', 'image_15', 'image_16', 'image_17', 'image_18', 'image_19', 'image_20', 'image_21', 'image_22', 'image_23', 'image_24'])

(1, 256, 256)

In [50]:
importlib.reload(predict)
output = predict.run("data/Brain-Tumor-Progression", "ProtocolName")
output["prediction"] = output["prediction"]
output["label"] = output["label"]
pd.set_option('display.max_rows', 500)
#display(pd.DataFrame(output))

output_df = predict.get_majority_vote(output)
display(output_df.head(20))
display(output_df.groupby("prediction").agg({"label": lambda x: list(x)}).reset_index())

Found DICOM images
Data already converted.
Create dataset
Loading cached output


,label,prediction,count
0,b'B Ax T1 +C',0,30
7,b'B T1 AXIAL SE +',1,78
12,b'Brain RCBV',1,15
17,b'PWI 8CH RCBV/15',1,20


,prediction,label
0,0,[b'B Ax T1 +C']
1,1,"[b'B T1 AXIAL SE +', b'Brain RCBV', b'PWI 8CH ..."


In [ ]:
importlib.reload(predict)
filter = predict.construct_regex(output_df)
display(filter)

In [ ]:
with open('config.yaml', 'r') as ymlfile:
    config = yaml.load(ymlfile, Loader=Loader)
config

In [8]:
with open("/home/raffi/Projects/DeepDicomSortOwn/data/output/Predictions_model_all_brain_tumor_data.csv", newline='') as f:
    reader = csv.reader(f, delimiter="\t")
    output = list(reader)
display(output[1])

with open("/home/raffi/Projects/DeepDicomSortOwn/data/output/Predictions_model_all_brain_tumor_data.csv", newline='') as f:
    df = pd.read_csv(f, delimiter="\t")
display(df.head(20))

['/input/NIFTI_SLICES/PGBM-018__19930218__MR__1.3.6.1.4.1.14519.5.2.1.4429.7055.816964431247986743876504511902__1.3.6.1.4.1.14519.5.2.1.4429.7055.264938771600653383755230242737_24.nii.gz',
 '2',
 '0']

,/input/NIFTI_SLICES/PGBM-011__19890629__MR__1.3.6.1.4.1.14519.5.2.1.4429.7055.223438270011735412196259897437__1.3.6.1.4.1.14519.5.2.1.4429.7055.418756081736560414491870730948_3.nii.gz,2,0
0,/input/NIFTI_SLICES/PGBM-018__19930218__MR__1....,2,0
1,/input/NIFTI_SLICES/PGBM-006__19961210__MR__1....,2,0
2,/input/NIFTI_SLICES/PGBM-019__19911002__MR__1....,1,0
3,/input/NIFTI_SLICES/PGBM-018__19930218__MR__1....,2,0
4,/input/NIFTI_SLICES/PGBM-015__19920512__MR__1....,2,0
5,/input/NIFTI_SLICES/PGBM-005__19910702__MR__1....,2,0
6,/input/NIFTI_SLICES/PGBM-011__19890824__MR__1....,2,0
7,/input/NIFTI_SLICES/PGBM-011__19890629__MR__1....,2,0
8,/input/NIFTI_SLICES/PGBM-006__19961210__MR__1....,2,0
9,/input/NIFTI_SLICES/PGBM-018__19930415__MR__1....,2,0
